# Getting started with IXPE data 

## 1. Introduction

This notebook is a tutorial on accessing IXPE data on Sciserver and getting started with analysing them. You will learn to download the data, extract the source and background regions and perform spectro-polarimetric fits.

It also highly recommended that new users read the IXPE Quick Start Guide ([linked here](https://heasarc.gsfc.nasa.gov/docs/ixpe/analysis/IXPE_quickstart.pdf)) and the recommended practices for statistical treatment of IXPE results [here](https://heasarcdev.gsfc.nasa.gov/docs/ixpe/analysis/IXPE_Stats-Advice.pdf).

<div style='color: #333; background: #ffffdf; padding:20px; border: 4px solid #fadbac'>
<b>Running On Sciserver:</b><br>
When running this notebook inside Sciserver, make sure the HEASARC data drive is mounted when initializing the Sciserver compute container. <a href='https://heasarc.gsfc.nasa.gov/docs/sciserver/'>See details here</a>.
<br>
Also, this notebook requires <code>heasoftpy</code>, which is available in the (heasoft) conda environment. You should see (heasoft) at the top right of the notebook. If not, click there and select it.

<b>Running Outside Sciserver:</b><br>
If running outside Sciserver, some changes will be needed, including:<br>
&bull; Make sure heasoftpy and heasoft are installed (<a href='https://heasarc.gsfc.nasa.gov/docs/software/lheasoft/'>Download and Install heasoft</a>).<br>
&bull; Unlike on Sciserver, where the data is available locally, you will need to download the data to your machine.<br>
</div>

## 2. Module Imports
We need the following python modules:

<div style='color: #333; background: #ffffdf; padding:20px; border: 4px solid #fadbac'>
In this example, reprocessing the data is not required. Instead the level 2 data products are sufficient. If you need to reprocess the data, the IXPE tools are available with <code>from heasoftpy import ixpe</code>.
</div>

In [1]:
import glob
import matplotlib.pyplot as plt
import heasoftpy as hsp
from heasoftpy import ixpe
import xspec

## 3. Finding the Data

On Sciserver, all the HEASARC data ire mounted locally under /FTP/, so once we have the path to the data, we can directly access it without the need to download it.

For our exploratory data analysis, we will use an observation of the blazar Mrk 501 (ObsID 01004701).

For running this locally, please download the data (e.g by using the wget command)

```
wget -q -nH --no-check-certificate --cut-dirs=5 -r -l0 -c -N -np -R 'index*' -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/ixpe/data/obs/01//01004701/
```

In [2]:
#data_path = "/FTP/ixpe/data/obs/01/01004701" ###this is the path on Sciserver
data_path = "./01004701"

Check the contents of this folder

It should contain the standard IXPE data files, which include:
   - `event_l1` and `event_l2`: level 1 and 2 event files, respectively.
   - `auxil`: auxiliary data files, such as exposure maps.
   - `hk`: house-keeping data such as orbit files etc.
    
For a complete description of data formats of the level 1, level 2 and calibration data products, see the support documentation on the [IXPE Website](https://heasarc.gsfc.nasa.gov/docs/ixpe/analysis/#supportdoc)

In [3]:
glob.glob(f'{data_path}/*')

['./01004701/hk',
 './01004701/auxil',
 './01004701/event_l1',
 './01004701/event_l2']

## 4. Exploring The Data
To Analyze the data within the notebook, we use heasoftpy.

In the folder for each observation, check for a README file. This file is included with a description of known issues (if any) with the processing for that observation.

In this IXPE example, it is not necessary to reprocess the data. Instead the level 2 data products can be analysed directly.

In [4]:
# set some input
indir  = data_path
obsid  = indir.split('/')[-1] 

filelist = glob.glob(f'{indir}/event_l2/*')
filelist

['./01004701/event_l2/ixpe01004701_det3_evt2_v01.fits.gz',
 './01004701/event_l2/ixpe01004701_det2_evt2_v01.fits.gz',
 './01004701/event_l2/ixpe01004701_det1_evt2_v01.fits.gz']

We see that there are three files: one event file for each detector. We can examine the structure of these level 2 files.

In [5]:
det3_fits = filelist[0]
det2_fits = filelist[1]
det1_fits = filelist[2]

#print the file structure for event 1 detector file
out = hsp.fstruct(infile=det1_fits).stdout
print(out)

  No. Type     EXTNAME      BITPIX Dimensions(columns)      PCOUNT  GCOUNT
 
   0  PRIMARY                  8     0                           0    1
   1  BINTABLE EVENTS          8     48(10) 101073               0    1
 
      Column Name                Format     Dims       Units     TLMIN  TLMAX
      1 TRG_ID                     J
      2 TIME                       D                   s
      3 STATUS                     16X
      4 STATUS2                    16X
      5 PI                         J                   chan          0      374
      6 W_MOM                      E
      7 X                          E                   pixel         1      600
      8 Y                          E                   pixel         1      600
      9 Q                          D
     10 U                          D
 
   2  BINTABLE GTI             8     16(2) 95                    0    1
 
      Column Name                Format     Dims       Units     TLMIN  TLMAX
      1 START         

## 5. Extracting the spectro polarimetric data 

### 5.1 Defining the Source and Background Regions

To obtain the source and background spectra from the Level 2 files, we need to define a source region and background region for the extraction. This can also be done using `ds9`. 

For the source, we extract a 60" circle centered on the source. For the background region, we use an annulus with inner radius of 132.000" and outer radius 252.000"

The region files should be independently defined for each telescope; in this example, the source location has the same celestial coordinates within 0.25" for all three detectors so a single source and a single background region can be used.

In [31]:
f = open("src.reg", "w")
f.write('circle(16:53:51.766,+39:45:44.41,60.000")')
f.close()

f = open("bkg.reg", "w")
f.write('annulus(16:53:51.766,+39:45:44.41,132.000",252.000")')
f.close()

### 5.2 Running the extractor tools

The `extractor` tool from FTOOLS, can now be used to extract I,Q and U spectra from IXPE Level 2
event lists as shown below. 

The help for the tool can be displayed using the `hsp.extractor?` command. 

First, we extract the source I,Q and U spectra 

In [6]:
#Extract source I,Q and U spectra for DU1
out = hsp.extractor(filename=det1_fits, binlc=10.0, eventsout='NONE', imgfile='NONE',fitsbinlc='NONE', 
              phafile= 'ixpe_det1_src_.pha',regionfile='src.reg', timefile='NONE', stokes='NEFF', 
              polwcol='W_MOM', tcol='TIME', ecol='PI', xcolf='X', xcolh='X',ycolf='Y', ycolh='Y')
if out.returncode != 0:
    print(out.stdout)
    raise Exception('extractor for det1 failed!')

#Extract source I,Q and U spectra for DU2
out = hsp.extractor(filename=det2_fits, binlc=10.0, eventsout='NONE', imgfile='NONE',fitsbinlc='NONE', 
              phafile= 'ixpe_det2_src_.pha',regionfile='src.reg', timefile='NONE', stokes='NEFF', 
              polwcol='W_MOM', tcol='TIME', ecol='PI', xcolf='X', xcolh='X',ycolf='Y', ycolh='Y')
if out.returncode != 0:
    print(out.stdout)
    raise Exception('extractor for det2 failed!')

#Extract source I,Q and U spectra for DU3
out = hsp.extractor(filename=det3_fits, binlc=10.0, eventsout='NONE', imgfile='NONE',fitsbinlc='NONE', 
              phafile= 'ixpe_det3_src_.pha',regionfile='src.reg', timefile='NONE', stokes='NEFF', 
              polwcol='W_MOM', tcol='TIME', ecol='PI', xcolf='X', xcolh='X',ycolf='Y', ycolh='Y')
if out.returncode != 0:
    print(out.stdout)
    raise Exception('extractor for det3 failed!')


Now repeat the process to extract background I,Q and U spectra

In [7]:
#Extract background I,Q and U spectra for DU1
out = hsp.extractor(filename=det1_fits, binlc=10.0, eventsout='NONE', imgfile='NONE',fitsbinlc='NONE', 
              phafile= 'ixpe_det1_bkg_.pha',regionfile='bkg.reg', timefile='NONE', stokes='NEFF', 
              polwcol='W_MOM', tcol='TIME', ecol='PI', xcolf='X', xcolh='X',ycolf='Y', ycolh='Y')
if out.returncode != 0:
    print(out.stdout)
    raise Exception('extractor for det1 failed!')

#Extract background I,Q and U spectra for DU2
out = hsp.extractor(filename=det2_fits, binlc=10.0, eventsout='NONE', imgfile='NONE',fitsbinlc='NONE', 
              phafile= 'ixpe_det2_bkg_.pha',regionfile='bkg.reg', timefile='NONE', stokes='NEFF', 
              polwcol='W_MOM', tcol='TIME', ecol='PI', xcolf='X', xcolh='X',ycolf='Y', ycolh='Y')
if out.returncode != 0:
    print(out.stdout)
    raise Exception('extractor for det2 failed!')

#Extract background I,Q and U spectra for DU3
out = hsp.extractor(filename=det3_fits, binlc=10.0, eventsout='NONE', imgfile='NONE',fitsbinlc='NONE', 
              phafile= 'ixpe_det3_bkg_.pha',regionfile='bkg.reg', timefile='NONE', stokes='NEFF', 
              polwcol='W_MOM', tcol='TIME', ecol='PI', xcolf='X', xcolh='X',ycolf='Y', ycolh='Y')
if out.returncode != 0:
    print(out.stdout)
    raise Exception('extractor for det3 failed!')

### 5.3 Obtaining the Response Files
For the I spectra, you will need to include the RMF (Response Matrix File), and the ARF (Ancillary Response File).

For the Q and U spectra, you will need to include the RMF and MRF (Modulation Response File). The MRF is defined by the product of the energy-dependent modulation factor, 
(E) and the ARF.

The location of the calibration files can be obtained through the hsp.quzcif tool. Type in hsp.quzcif? to get more information on this function.

Note that the output of the hsp.quzcif gives the path to more than one file. This is because there are 3 sets of response files, corresponding to the different weighting schemes.

For the 'NEFF' weighting, use 'alpha07_vv'.
For the 'SIMPLE' weighting, use 'alpha075simple_vv'.
For the 'UNWEIGHTED' version, use '20170101_vv'.
Where vv is the version number of the response files. The use of the latest version of the files is recommended.

In following, we use vv = 02 for the RMF.

In [9]:
hsp.quzcif?

In [8]:
# get the on-axis rmf
res = hsp.quzcif(mission='ixpe', instrument='gpd',detector='DU1',
             filter='-', date='-', time='-',expr='-',codename='MATRIX')

rmf1 = [x.split()[0] for x in res.output if 'alpha075_02'  in x][0]

res = hsp.quzcif(mission='ixpe', instrument='gpd',detector='DU2',
             filter='-', date='-', time='-',expr='-',codename='MATRIX')

rmf2 = [x.split()[0] for x in res.output if 'alpha075_02'  in x][0]

res = hsp.quzcif(mission='ixpe', instrument='gpd',detector='DU3',
             filter='-', date='-', time='-',expr='-',codename='MATRIX')

rmf3 = [x.split()[0] for x in res.output if 'alpha075_02'  in x][0]

#### 5.31 Generating ARF and MRF using _ixpecalcarf_

For the Q and U spectra, you will need to include the RMF and MRF (Modulation Response File). The MRF is defined by the product of the energy-dependent modulation factor, $\mu$(E) and the ARF.

These files can be generated for each observation using the _ixpecalcarf_ tool. _ixpecalcarf_ correctly accounts for off-axis vignetting and source extraction radius, and the use of this tool is recommended over utilizing arf and mrf files from caldb.  

The spacecraft attitude file, part of the engineering housekeeping file suite retrievable from the “hk” subdirectory in the data archive is required to run this. The resulting Response File is appropriate for an observation in which IXPE has been pointed so that the sky location of the target is at the center of the detector, and a 1.0 source extraction region radius (radius=1.0), and NEFF weighting method (weight=1) have been used to extract the spectrum file. (Other weighting options are weight=0 for UNWEIGHTED and weight=2 for SIMPLE)

The generated MRF files can be used for both the Q and U spectra

In [35]:
#specify attitude files

attfile_d1 = glob.glob(f'{data_path}/hk/ixpe*_det1_att_v*.fits.gz')
attfile_d2 = glob.glob(f'{data_path}/hk/ixpe*_det2_att_v*.fits.gz')
attfile_d3 = glob.glob(f'{data_path}/hk/ixpe*_det3_att_v*.fits.gz')

In [29]:
# generate the mrf for Q and U spectra
ixpe.ixpecalcarf(evtfile=det1_fits , attfile=attfile_d1[0], specfile='none',
                 radius=1.0, arfout='ixpe_det1_src.mrf', weight=1, resptype='mrf', clobber='yes')

mrf1 = 'ixpe_det1_src.mrf'

ixpe.ixpecalcarf(evtfile=det2_fits , attfile=attfile_d2[0], specfile='none',
                 radius=1.0, arfout='ixpe_det2_src.mrf', weight=1, resptype='mrf', clobber='yes')

mrf2 = 'ixpe_det2_src.mrf'

ixpe.ixpecalcarf(evtfile=det3_fits , attfile=attfile_d3[0], specfile='none',
                 radius=1.0, arfout='ixpe_det3_src.mrf', weight=1, resptype='mrf', clobber='yes')

mrf3 = 'ixpe_det3_src.mrf'

---------------------
:: Execution Result ::
---------------------
> Return Code: 1
> Output:
Error: Did not find a valid UVFILT file from the CALDB. The file 'https://heasarc.gsfc.nasa.gov/FTP/caldb/data/ixpe/gpd/bcf/uvfilt/ixpe_d3_20170101_uvfilt_01.fits' does not exist.

> Parameters:
	evtfile   : data/obs/01/01004701/event_l2/ixpe01004701_det3_evt2_v01.fits.gz
	attfile   : data/obs/01/01004701/hk/ixpe01004701_det1_att_v01.fits.gz
	resptype  : mrf
	specfile  : none
	vignfile  : caldb
	reeffile  : caldb
	uvfilt    : caldb
	grfilt    : caldb
	axeffa    : caldb
	modfact   : caldb
	qefile    : caldb
	radius    : 1.0
	offarcmin : -1.0
	weight    : 1
	detfilter : 0
	arfin     : 
	ox        : 0.0
	oy        : 0.0
	arfout    : ixpe_det1_src.mrf
	clobber   : yes
	mode      : ql

A similar ixpecalcarf command can be used to generate an ARF for Stokes I spectra by setting resptype=arf.

Note that, in a typical XSPEC implementation, properly-scaled background spectra are simply subtracted from the observed spectrum to obtain the source spectrum so that instrument response files are not needed for the background spectra.

In [15]:
# generate the arf for I spectra

ixpe.ixpecalcarf(evtfile=det1_fits , attfile=attfile_d1[0], specfile='none',
                 radius=1.0, arfout='ixpe_det1_src.arf', weight=1, resptype='arf', clobber='yes')

arf1 = 'ixpe_det1_src.arf'

ixpe.ixpecalcarf(evtfile=det2_fits , attfile=attfile_d2[0], specfile='none',
                 radius=1.0, arfout='ixpe_det2_src.arf', weight=1, resptype='arf', clobber='yes')

arf2 = 'ixpe_det2_src.arf'

ixpe.ixpecalcarf(evtfile=det3_fits , attfile=attfile_d3[0], specfile='none',
                 radius=1.0, arfout='ixpe_det3_src.arf', weight=1, resptype='arf', clobber='yes')

arf3 = 'ixpe_det3_src.arf'

---------------------
:: Execution Result ::
---------------------
> Return Code: 1
> Output:
Error: Did not find a valid UVFILT file from the CALDB. The file 'https://heasarc.gsfc.nasa.gov/FTP/caldb/data/ixpe/gpd/bcf/uvfilt/ixpe_d1_20170101_uvfilt_01.fits' does not exist.

> Parameters:
	evtfile   : data/obs/01/01004701/event_l2/ixpe01004701_det1_evt2_v01.fits.gz
	attfile   : data/obs/01/01004701/hk/ixpe01004701_det1_att_v01.fits.gz
	resptype  : arf
	specfile  : none
	vignfile  : caldb
	reeffile  : caldb
	uvfilt    : caldb
	grfilt    : caldb
	axeffa    : caldb
	modfact   : caldb
	qefile    : caldb
	radius    : 1.0
	offarcmin : -1.0
	weight    : 1
	detfilter : 0
	arfin     : 
	ox        : 0.0
	oy        : 0.0
	arfout    : ixpe_det1_src.arf
	clobber   : no
	mode      : ql

### 5.4 Load data into PyXSPEC and start fitting 

In [19]:
rmf_list = [rmf1,rmf2,rmf3]
mrf_list = [mrf1,mrf2,mrf3]
arf_list = [arf1,arf2,arf3]
du_list = [1,2,3]

xspec.AllData.clear()

x=0 #factor to get the spectrum numbering right 
for (du, rmf_file, mrf_file, arf_file) in zip(du_list, rmf_list, mrf_list, arf_list):

    #Load the I data
    xspec.AllData("%i:%i ixpe_det%i_src_I.pha"%(du, du+x, du))
    xspec.AllData(f"{du}:{du+x} ixpe_det{du}_src_I.pha")
    s = xspec.AllData(du+x)
    
    # #Load response and background files
    s.response = rmf_file
    s.response.arf = arf_file
    s.background = 'ixpe_det%i_bkg_I.pha'%du
    
    #Load the Q data
    xspec.AllData("%i:%i ixpe_det%i_src_Q.pha"%(du, du+x+1, du))
    s = xspec.AllData(du+x+1)
    
    # #Load response and background files
    s.response = rmf_file
    s.response.arf = mrf_file
    s.background = 'ixpe_det%i_bkg_Q.pha'%du
    
    #Load the U data
    xspec.AllData("%i:%i ixpe_det%i_src_U.pha"%(du, du+x+2, du))
    s = xspec.AllData(du+x+2)
    
    # #Load response and background files
    s.response = rmf_file
    s.response.arf = mrf_file
    s.background = 'ixpe_det%i_bkg_U.pha'%du
    
    x+=2



1 spectrum  in use
 
Spectral Data File: ixpe_det1_src_I.pha  Spectrum 1
Net count rate (cts/s) for Spectrum:1  3.744e-01 +/- 1.960e-03
 Assigned to Data Group 1 and Plot Group 1
  Noticed Channels:  1-375
  Telescope: IXPE Instrument: GPD  Channel Type: PI
  Exposure Time: 9.782e+04 sec
  Filtering Keys: 
    Stokes: 0
 Using fit statistic: chi
 No response loaded.

***Warning!  One or more spectra are missing responses,
               and are not suitable for fit.

Spectrum #: 1 replaced 

1 spectrum  in use
 
Spectral Data File: ixpe_det1_src_I.pha  Spectrum 1
Net count rate (cts/s) for Spectrum:1  3.744e-01 +/- 1.960e-03
 Assigned to Data Group 1 and Plot Group 1
  Noticed Channels:  1-375
  Telescope: IXPE Instrument: GPD  Channel Type: PI
  Exposure Time: 9.782e+04 sec
  Filtering Keys: 
    Stokes: 0
 Using fit statistic: chi
 No response loaded.

***Warning!  One or more spectra are missing responses,
               and are not suitable for fit.
New filename ( "none" or "/*" t

Error: cannot read response file https://heasarc.gsfc.nasa.gov/FTP/caldb/data/ixpe/gpd/cpf/rmf/ixpe_d1_20170101_alpha075_02.rmf
terminated at user request


Exception: Error: No response is assigned to source 1 for spectrum 1

In [ ]:
#Ignore all channels except 2-8keV
xspec.AllData.ignore("0.0-2.0, 8.0-**")

In [ ]:
model = xspec.Model('constant*tbabs(polconst*powerlaw)')

model.polconst.A = 0.05
model.polconst.psi = -50
model.TBabs.nH = 0.15
model.powerlaw.PhoIndex = 2.7
model.powerlaw.norm = 0.1

In [ ]:
m1 = xspec.AllModels(1)
m2 = xspec.AllModels(2)
m3 = xspec.AllModels(3)

m1.constant.factor = 1.0
m1.constant.factor.frozen = True
m2.constant.factor = 0.8
m3.constant.factor = 0.9

In [ ]:
xspec.AllModels.show()

In [ ]:
xspec.Fit.perform()

### 5.5 Plotting the results

This is done through `matplotlib`.

In [ ]:
xspec.Plot.area=True
xspec.Plot.xAxis='keV'
xspec.Plot('lda')
yVals=xspec.Plot.y()
yErr = xspec.Plot.yErr()
xVals = xspec.Plot.x()
xErr = xspec.Plot.xErr()
mop = xspec.Plot.model()


fig, ax = plt.subplots(figsize=(10,6))
ax.errorbar(xVals, yVals, xerr=xErr, yerr=yErr, fmt='k.', alpha=0.2)
ax.plot(xVals, mop,'r-')
ax.set_xlabel('Energy (keV)')
ax.set_ylabel(r'counts/cm$^2$/s/keV')
ax.set_xscale("log")
ax.set_yscale("log")


In [ ]:
xspec.Plot.area=True
xspec.Plot.xAxis='keV'
xspec.Plot('polangle')
yVals=xspec.Plot.y()
yErr = [abs(y) for y in xspec.Plot.yErr()]
xVals = xspec.Plot.x()
xErr = xspec.Plot.xErr()
mop = xspec.Plot.model()


fig, ax = plt.subplots(figsize=(10,6))
ax.errorbar(xVals, yVals, xerr=xErr, yerr=yErr, fmt='k.', alpha=0.2)
ax.plot(xVals, mop,'r-')
ax.set_xlabel('Energy (keV)')
ax.set_ylabel(r'Polangle')

## 6. Interpreting the results from XSPEC

There are two parameters of interest in our example. These given by the polarization fraction, A,
and polarization angle, $\psi$. The XSPEC error (or uncertainty) command can be used
to deduce confidence intervals for these parameters. 

We can estimate the 99% confidence interval for these two parameters.

In [ ]:
xspec.Fit.error("6.635 3") #Uncertainty on parameter 3

In [ ]:
xspec.Fit.error("6.635 4") #Uncertainty on parameter 4

Of particular interest is the 2-D error contour for the polarization fraction and polarization angle.

In [ ]:
lch = xspec.Xset.logChatter
xspec.Xset.logChatter = 20

# Create and open a log file for XSPEC output. 
# This step can sometimes take a few minutes. Please be patient!
logFile = xspec.Xset.openLog("steppar.txt")

xspec.Fit.steppar("3 0.00 0.21 41 4 -90 0 36")

# Close XSPEC's currently opened log file.
xspec.Xset.closeLog()

In [ ]:
#Plot the results
xspec.Plot.area=True
xspec.Plot('contour ,,4 1.386, 4.61 9.21 13.81')
yVals=xspec.Plot.y()
xVals = xspec.Plot.x()
zVals = xspec.Plot.z()
levelvals = xspec.Plot.contourLevels()
statval = xspec.Fit.statistic
plt.contour(xVals,yVals,zVals,levelvals)
plt.ylabel('Psi (deg)')
plt.xlabel('A')
plt.errorbar(m1.polconst.A.values[0],m1.polconst.psi.values[0],fmt='+')

Note that the detection is deemed "highly probable" (confidence C > 99.9%) as
A/$\sigma$ = 4.123 >
$\sqrt(-2 ln(1- C)$ where $\sigma$ = 0.01807 as given by XSPEC above. 

Finally, we can use PIMMS to estimate the Minimum Detectable Polarization (MDP). 

To do this, we first use XSPEC to determine the (model) flux on the 2-8 keV energy range:

In [ ]:
xspec.AllModels.calcFlux("2.0 8.0")

Then enter the appropriate parameters (power law model with Galactic hydrogen column density
$n_H/10^{22}$ = 0.449, photon index $\Gamma$ = 2.59, 
and flux (average of three detectors) 8.45 x $10^{-11} erg cm^{-2} s^{-1}$ in the 2-8 keV range) into [PIMMS](https://heasarc.gsfc.nasa.gov/cgi-bin/Tools/w3pimms/w3pimms.pl). 

PIMMS returns MDP99 of 5.35% for a 100 ks exposure. Scaling by the actual
mean of exposure time of 97243 s gives an MDP99 of 5.50% meaning that, for an unpolarized source with these physical parameters, an IXPE observation will return a value A > 0.05 only 1% of the time. 

This is consistent with the highly probable detection deduced here of a polarization fraction of 7.45$\pm$1.8%.

## Additional Resources

Visit the IXPE [GOF Website](https://heasarc.gsfc.nasa.gov/docs/ixpe/analysis/) and the IXPE [Project Website at MSFC](https://ixpe.msfc.nasa.gov/for_scientists/index.html) for more resources.